In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import os
from time import gmtime, strftime
import torch.nn.functional as F
from tqdm import tqdm

# Load the tokenizer and model for sentiment analysis
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")

# Move the model to GPU if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# File paths
input_file = "final_dataset.csv"  # Input CSV file
output_file = "single_model_output.csv"  # Output CSV file
error_indices_file = "error_indices.txt"

# Load the input data
df = pd.read_csv(input_file, encoding='utf-8')

# Ensure the output columns exist
if 'label' not in df.columns:
    df['label'] = ''
if 'confidence' not in df.columns:
    df['confidence'] = ''

# Function for sentiment classification with confidence
def classify_text_with_confidence(text):
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        outputs = model(**inputs)
        probabilities = F.softmax(outputs.logits, dim=1)  # Get softmax probabilities
        predicted_class = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0, predicted_class].item()
        return predicted_class, confidence
    except Exception as e:
        print(f"Error during classification: {e}")
        return None, None

error_indices = []

# Process the data with tqdm for progress tracking
for i in tqdm(range(len(df)), desc="Processing rows"):
    try:
        text = df.iloc[i]['content']
        if pd.isna(text) or text.strip() == "":
            print(f"Skipping empty content at index {i}")
            continue
        label, confidence = classify_text_with_confidence(text)
        if label is None:
            error_indices.append(i)
            print(f"Classification failed at index {i}, added to error list.")
            continue
        df.loc[i, 'label'] = label
        df.loc[i, 'confidence'] = confidence
        print(f"Text '{i}' Classified as {label} with confidence {confidence:.4f}.")
    except Exception as e:
        print(f"Error at index {i}: {e}")
        error_indices.append(i)
        continue

# Save the results
df.to_csv(output_file, index=False)

# Save error indices
if error_indices:
    with open(error_indices_file, "w") as f:
        for index in error_indices:
            f.write(f"{index}\n")

# End time
end_time = strftime("%Y-%m-%d %H:%M:%S", gmtime())
print("Processing done!")
print(f"Indices with classification errors saved to {error_indices_file}.")


Processing rows:   0%|          | 6/26150 [00:00<22:41, 19.20it/s]  

Text '0' Classified as 0 with confidence 0.6207.
Text '1' Classified as 2 with confidence 0.6812.
Text '2' Classified as 1 with confidence 0.4367.
Text '3' Classified as 0 with confidence 0.7548.
Text '4' Classified as 0 with confidence 0.4984.
Text '5' Classified as 1 with confidence 0.4467.
Text '6' Classified as 1 with confidence 0.3819.
Text '7' Classified as 2 with confidence 0.3628.
Text '8' Classified as 2 with confidence 0.3433.
Text '9' Classified as 1 with confidence 0.3648.
Text '10' Classified as 0 with confidence 0.4203.
Text '11' Classified as 1 with confidence 0.4961.


Processing rows:   0%|          | 18/26150 [00:00<12:12, 35.69it/s]

Text '12' Classified as 1 with confidence 0.6424.
Text '13' Classified as 1 with confidence 0.4791.
Text '14' Classified as 2 with confidence 0.4505.
Text '15' Classified as 0 with confidence 0.4079.
Text '16' Classified as 2 with confidence 0.6870.
Text '17' Classified as 0 with confidence 0.3785.
Text '18' Classified as 1 with confidence 0.4141.
Text '19' Classified as 2 with confidence 0.4653.
Text '20' Classified as 1 with confidence 0.8905.
Text '21' Classified as 1 with confidence 0.4176.


Processing rows:   0%|          | 31/26150 [00:00<09:24, 46.25it/s]

...

Processing rows: 100%|█████████▉| 26131/26150 [12:44<00:00, 38.35it/s]

Text '26121' Classified as 1 with confidence 0.5713.
Text '26122' Classified as 1 with confidence 0.4116.
Text '26123' Classified as 0 with confidence 0.6124.
Text '26124' Classified as 0 with confidence 0.6083.
Text '26125' Classified as 1 with confidence 0.3901.
Text '26126' Classified as 1 with confidence 0.5028.
Text '26127' Classified as 0 with confidence 0.4148.
Text '26128' Classified as 1 with confidence 0.5912.
Text '26129' Classified as 2 with confidence 0.7344.
Text '26130' Classified as 1 with confidence 0.3560.
Text '26131' Classified as 1 with confidence 0.6028.
Text '26132' Classified as 0 with confidence 0.3457.
Text '26133' Classified as 1 with confidence 0.5854.
Text '26134' Classified as 2 with confidence 0.3688.
Text '26135' Classified as 2 with confidence 0.4914.


Processing rows: 100%|██████████| 26150/26150 [12:45<00:00, 34.18it/s]

Text '26136' Classified as 1 with confidence 0.4743.
Text '26137' Classified as 0 with confidence 0.4275.
Text '26138' Classified as 2 with confidence 0.5355.
Text '26139' Classified as 0 with confidence 0.4293.
Text '26140' Classified as 1 with confidence 0.4290.
Text '26141' Classified as 1 with confidence 0.6301.
Text '26142' Classified as 0 with confidence 0.5014.
Text '26143' Classified as 0 with confidence 0.4598.
Text '26144' Classified as 2 with confidence 0.4169.
Text '26145' Classified as 1 with confidence 0.4840.
Text '26146' Classified as 1 with confidence 0.7030.
Text '26147' Classified as 1 with confidence 0.4408.
Text '26148' Classified as 1 with confidence 0.4122.
Text '26149' Classified as 2 with confidence 0.4123.


Processing done!
Indices with classification errors saved to error_indices.txt.
